## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-11-11-50-01 +0000,bbc,"War crimes being committed in Sudan, ICC believes",https://www.bbc.com/news/articles/cvg819yxkp5o
1,2025-07-11-11-43-51 +0000,nyt,Good Vibes in Trump Trade Talks Haven’t Shield...,https://www.nytimes.com/2025/07/11/world/canad...
2,2025-07-11-11-42-46 +0000,bbc,Unite votes to suspend Rayner's membership ove...,https://www.bbc.com/news/articles/cx24de0d9rdo
3,2025-07-11-11-30-00 +0000,nypost,East Northport’s Preston Carey excited to play...,https://nypost.com/2025/07/11/sports/east-nort...
4,2025-07-11-11-21-25 +0000,nyt,Behind Trump’s Decision to Tax Brazil to Save ...,https://www.nytimes.com/2025/07/11/world/ameri...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2333/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,13
36,brazil,6
172,uk,5
38,bolsonaro,4
14,canada,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
61,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,39
53,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,35
35,2025-07-11-02-31-43 +0000,nypost,Trump admin still deciding where it would depo...,https://nypost.com/2025/07/10/us-news/trump-ad...,28
44,2025-07-10-20-22-02 +0000,nyt,US-Brazil Tariffs: What to Know About Trump’s ...,https://www.nytimes.com/2025/07/10/world/ameri...,28
47,2025-07-10-18-19-52 +0000,bbc,Why is Trump targeting Brazil - and will it ba...,https://www.bbc.com/news/articles/crk68drj57mo,28


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
61,39,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...
20,27,2025-07-11-08-22-06 +0000,nypost,Georgia teen Sarah Grace Patrick accused of ki...,https://nypost.com/2025/07/11/us-news/georgia-...
21,18,2025-07-11-08-06-08 +0000,bbc,UK-France migrant deal 'robust' against legal ...,https://www.bbc.com/news/articles/cx24d70gw41o
53,18,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
37,17,2025-07-11-01-57-20 +0000,nypost,Florida scammer posing as doctor gave Botox in...,https://nypost.com/2025/07/10/us-news/florida-...
5,15,2025-07-11-11-21-07 +0000,nypost,Infamous ‘tot mom’ Casey Anthony spotted cozyi...,https://nypost.com/2025/07/11/us-news/casey-an...
13,15,2025-07-11-09-30-00 +0000,wsj,Nuclear power exemplifies how revamping dated ...,https://www.wsj.com/economy/trumps-unsung-econ...
33,15,2025-07-11-03-35-21 +0000,nypost,Grieving mom of dead Camp Mystic camper desper...,https://nypost.com/2025/07/10/us-news/grieving...
38,13,2025-07-11-01-41-11 +0000,nypost,"NYPD, other city agencies crack down on trash,...",https://nypost.com/2025/07/10/us-news/nyc-crac...
32,13,2025-07-11-04-01-24 +0000,nypost,Protester fires gun at federal agents in chaot...,https://nypost.com/2025/07/11/us-news/californ...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
